In [1]:
import pandas as pd
import numpy as np
import sys
import warnings
warnings.filterwarnings('ignore')
import os
os.chdir('/Users/veronica/Documents/GitHub/analysis')
from src import utils, credentials

In [2]:
creds = credentials.creds
engine = utils.create_sql_db(creds)

## Data

In [3]:
query = """WITH merchant_share AS (
SELECT 
    store_id,
    plan_id,
    pricing_model,
    merchant_revenue_share,
    start_date,
    end_date
FROM analytics.merchant_rev_share_facts mrs
GROUP BY 1,2,3,4,5,6
ORDER BY 1
),

base AS (
SELECT 
  ue.contract_id,
  ue.store_id,
  ue.seller_name,
  ue.plan_id,
  ue.plan_transaction_date,
  ue.product_purchase_price,
  ue.plan_purchase_price,
  ue.premium,
  ue.merchant_margin_usd,
  ue.plan_sku,
  ue.contract_status,
  c.product_id,
  prod.reimbursement_amount/100.0 as reimbursement_amount,
  prod.price as price,
  CASE WHEN prod.reimbursement_amount IS NOT NULL THEN prod.reimbursement_amount/100.0 ELSE prod.price END as price_final,
  m.merchant_cut,
  p.retail_target,
  p.fixed_price,
  p.price_high,
  p.price_low,
  ms.merchant_revenue_share/100.0 as merchant_revenue_share,
  ms.pricing_model
FROM analytics.contract_unit_econ_facts ue -- plan purchase price, premium
LEFT JOIN analytics.contract_facts c -- pull product_id
ON ue.contract_id = c.contract_id
LEFT JOIN analytics.merchant_facts m -- get merchant_cut used in offers API
ON ue.store_id = m.store_id
LEFT JOIN merchant_share ms
ON ue.store_id = ms.store_id AND ue.plan_id = ms.plan_id AND ue.plan_transaction_date BETWEEN ms.start_date AND ms.end_date
LEFT JOIN analytics.product_facts prod -- get reimbursement amount
ON c.product_id = prod.product_id AND ue.store_id = prod.store_id
LEFT JOIN analytics.plans_pricing p
ON ue.plan_id = p.plan_id
WHERE prod.price BETWEEN (p.price_low/100.0) and (p.price_high/100.0) 
AND ue.contract_status IN ('expired', 'live')
AND ue.seller_name NOT IN ('AAP', 'Peloton')
)

SELECT
  b.*,
  pm.cost/100.0 as cost 
FROM base b
LEFT JOIN analytics.plans_matching pm -- get cost (premium) based on price_final
ON b.plan_id = pm.plan_id AND b.plan_sku = pm.vendor_sku
WHERE 
b.price_final BETWEEN (b.price_low/100.0) AND (b.price_high/100.0)"""

df = pd.read_sql_query(query, engine)

In [4]:
# Clean Data
# Remove contracts with a fixed price, empty cost value, where cost=1, or product_price=1

In [4]:
df_clean = df[(df['cost'].notnull()) & (df['cost']!= 0.01) & 
              (df['fixed_price'].isnull()) & (df['price']!= 1)]

df_clean.head()

,contract_id,store_id,seller_name,plan_id,plan_transaction_date,product_purchase_price,plan_purchase_price,premium,merchant_margin_usd,plan_sku,...,price,price_final,merchant_cut,retail_target,fixed_price,price_high,price_low,merchant_revenue_share,pricing_model,cost
0,0e1b831d-1862-489e-ab75-c7b3a89755ef,0233ca5d-1b3f-4983-8b37-e3c503572f5a,T3,B0-HBGEN-1y,2021-05-30 21:18:12+00:00,74.99,8.49,3.27,4.245,EXTRPLBG014,...,89.0,89.0,0.5,0.09,NaN,8999,8000,0.5,Gross,3.27
1,94690dbc-7d82-459b-889f-79cf13403346,0233ca5d-1b3f-4983-8b37-e3c503572f5a,T3,B0-HBGEN-1y,2021-07-09 20:30:38+00:00,74.99,8.49,3.27,4.245,EXTRPLBG014,...,89.0,89.0,0.5,0.09,NaN,8999,8000,0.5,Gross,3.27
2,aa60f8ae-91e3-4e65-bcb5-6bf3b19ba14e,0233ca5d-1b3f-4983-8b37-e3c503572f5a,T3,B0-HBGEN-2y,2021-09-12 15:30:01+00:00,89.00,15.99,6.35,7.995,EXTRPLBG132,...,89.0,89.0,0.5,0.15,NaN,8999,8000,0.5,Gross,6.35
3,fafa06db-3292-487b-8e32-b2b1c51d59f1,0233ca5d-1b3f-4983-8b37-e3c503572f5a,T3,B0-HBGEN-3y,2021-08-14 01:16:57+00:00,74.99,23.99,9.49,11.995,EXTRPLBG250,...,89.0,89.0,0.5,0.20,NaN,8999,8000,0.5,Gross,9.49
4,6bcd872e-79ff-422d-887b-2c8d8db420f5,0233ca5d-1b3f-4983-8b37-e3c503572f5a,T3,B0-HBGEN-1y,2021-09-24 13:28:40+00:00,74.99,8.49,3.27,4.245,EXTRPLBG014,...,89.0,89.0,0.5,0.09,NaN,8999,8000,0.5,Gross,3.27


## Functions

In [5]:
# calculate target price of a product
def calc_target_price(df):
    df['target_price'] = df['retail_target'] * df['price']
    return df

In [6]:
# calculate minimum price for a product; price floor % is specified in the column name
def calc_min_price(df, extend_cut):
    df[f'min_price_{extend_cut}'] = df['cost'] / (1 - df['merchant_cut'] - extend_cut)
    return df

In [7]:
# calculate difference between target price and minimum price
def calc_diff(df, min_price_col):
    df['price_diff'] = abs(df['target_price'] - df[min_price_col])
    df['price_diff_pct'] = abs((df['target_price'] - df[min_price_col])/ df[min_price_col])
    return df

In [8]:
# create a "raw_price" col with max(target_price, minimum_price); 1/0 flag where min price > target price
def find_price_floor(df, extend_cut):
    df[f'raw_price_{extend_cut}'] = df[['target_price', f'min_price_{extend_cut}']].max(axis=1)
    df[f'price_floor_{extend_cut}'] = np.where(df[f'min_price_{extend_cut}'] > df['target_price'], 1, 0)
    return df

In [9]:
# apply rounding rules logic
def round_price(price):
    if price < 5:
        return round(price, 1) - 0.01
    if price < 10:
        return round(price/5, 1)*5 - 0.01
    if price < 50:
        return round(price) - 0.01
    if price < 100:
        return round(price/5)*5 - 0.01
    if price < 500: 
        return round(price/10)*10 - 1
    if price < 1000:
        return round(price/50)*50 - 1
    if price < 5000:
        return round(price/100)*100 - 1
    if price < 10000:
        return round(price/500)*500 - 1
    if price < 50000:
        return round(price/1000)*1000 - 1
    else:
        return np.nan

In [10]:
# calculate merchant margin for different plan prices (i.e when different price floor is used)
def merch_margin(x, plan_price_col):
    if x['pricing_model']=='Gross':
        return x[plan_price_col] * x['merchant_cut']
    if x['pricing_model']=='Net':
        return (x[plan_price_col] - x['premium']) * x['merchant_cut']
    else:
        return np.nan

## Analysis - Check how often price floor is triggered with 10% 

In [11]:
df_price = calc_target_price(df_clean)
df_price = calc_min_price(df_price, 0.10)
df_price = calc_diff(df_price, 'min_price_0.1')
df_price = find_price_floor(df_price, 0.10)
df_price.head()

,contract_id,store_id,seller_name,plan_id,plan_transaction_date,product_purchase_price,plan_purchase_price,premium,merchant_margin_usd,plan_sku,...,price_low,merchant_revenue_share,pricing_model,cost,target_price,min_price_0.1,price_diff,price_diff_pct,raw_price_0.1,price_floor_0.1
0,0e1b831d-1862-489e-ab75-c7b3a89755ef,0233ca5d-1b3f-4983-8b37-e3c503572f5a,T3,B0-HBGEN-1y,2021-05-30 21:18:12+00:00,74.99,8.49,3.27,4.245,EXTRPLBG014,...,8000,0.5,Gross,3.27,8.01,8.175,0.165,0.020183,8.175,1
1,94690dbc-7d82-459b-889f-79cf13403346,0233ca5d-1b3f-4983-8b37-e3c503572f5a,T3,B0-HBGEN-1y,2021-07-09 20:30:38+00:00,74.99,8.49,3.27,4.245,EXTRPLBG014,...,8000,0.5,Gross,3.27,8.01,8.175,0.165,0.020183,8.175,1
2,aa60f8ae-91e3-4e65-bcb5-6bf3b19ba14e,0233ca5d-1b3f-4983-8b37-e3c503572f5a,T3,B0-HBGEN-2y,2021-09-12 15:30:01+00:00,89.00,15.99,6.35,7.995,EXTRPLBG132,...,8000,0.5,Gross,6.35,13.35,15.875,2.525,0.159055,15.875,1
3,fafa06db-3292-487b-8e32-b2b1c51d59f1,0233ca5d-1b3f-4983-8b37-e3c503572f5a,T3,B0-HBGEN-3y,2021-08-14 01:16:57+00:00,74.99,23.99,9.49,11.995,EXTRPLBG250,...,8000,0.5,Gross,9.49,17.80,23.725,5.925,0.249737,23.725,1
4,6bcd872e-79ff-422d-887b-2c8d8db420f5,0233ca5d-1b3f-4983-8b37-e3c503572f5a,T3,B0-HBGEN-1y,2021-09-24 13:28:40+00:00,74.99,8.49,3.27,4.245,EXTRPLBG014,...,8000,0.5,Gross,3.27,8.01,8.175,0.165,0.020183,8.175,1


In [12]:
df_pf = df_price[df_price['price_floor_0.1']==1]

In [13]:
len(df_price)

241954

In [14]:
len(df_pf)

66545

In [15]:
len(df_pf['store_id'].unique())

167

In [16]:
price_col = ['seller_name', 'plan_id', 'plan_purchase_price', 'target_price', 'min_price_0.1', 'raw_price_10_rounded']

## Sanity Checks

In [17]:
# diff between plan purchase price and minimum price
df_pf['retail_min_diff'] = abs(df_pf['plan_purchase_price'] - df_pf['min_price_0.1'])

# diff between plan purchase price and target price
df_pf['retail_target_diff'] = abs(df_pf['plan_purchase_price'] - df_pf['target_price'])

# diff between minimum price and target price
df_pf['min_target_diff'] = abs(df_pf['min_price_0.1'] - df_pf['target_price'])

# ratio of min price to product price
df_pf['min_target'] = df_pf['min_price_0.1']/df_pf['price']

# diff between (min price/product price) and retail target
df_pf['target_diff'] = abs(df_pf['min_target'] - df_pf['retail_target'])

# round raw_price (raw price = max(min_price, target_price))
df_pf['raw_price_10_rounded'] = df_pf['raw_price_0.1'].apply(round_price)

# diff between plan purchase price and raw price
df_pf['retail_raw_rounded_diff'] = abs(df_pf['plan_purchase_price'] - df_pf['raw_price_10_rounded'])

### Note: there are some cases where `plan_purchase_price` doesn't correspond with `raw_price_rounded`

In [19]:
df_anom1 = df_pf[df_pf['retail_raw_rounded_diff'] > 5].sort_values(by='retail_raw_rounded_diff', ascending=False)
df_anom1[price_col+['retail_raw_rounded_diff', 'contract_id', 'retail_target']]

,seller_name,plan_id,plan_purchase_price,target_price,min_price_0.1,raw_price_10_rounded,retail_raw_rounded_diff,contract_id,retail_target
168535,Brilliant Earth,A1-AAJEW-LT,1499.00,159.5000,262.040000,259.00,1240.0,9e571642-a486-4d2c-8278-57824df45a43,0.11
22419,Brilliant Earth,A1-AAJEW-LT,1499.00,236.0000,492.120000,489.00,1010.0,52de6489-72bd-41c3-aadb-607caf1f1bf6,0.08
81491,Brilliant Earth,A1-AAJEW-LT,1499.00,247.2000,492.120000,489.00,1010.0,72fbdc3b-cc18-409e-81b3-ae3dfd8c06c1,0.08
253111,Brilliant Earth,A1-AAJEW-LT,899.00,130.9000,262.040000,259.00,640.0,d86dbcc6-a7b5-4ec4-8738-517536c0c141,0.11
81146,Brilliant Earth,A1-AAJEW-LT,899.00,141.9000,262.040000,259.00,640.0,b0097adb-65e3-4d26-bd6a-fd8981df19d0,0.11
...,...,...,...,...,...,...,...,...,...
332312,AutoAnything,10001-auto-part-base-replace-3y,30.99,30.4300,36.285714,35.99,5.0,7118c31b-586f-450f-baf2-9f93ddefad3e,0.17
142364,AutoAnything,B0-VEAPS-2y,29.99,29.1187,35.485714,34.99,5.0,02be7b3a-ca69-4556-a040-191071c88a6c,0.13
105382,AutoAnything,10001-auto-part-base-replace-3y,30.99,30.5915,36.285714,35.99,5.0,943868dd-4a2f-4535-9d3d-47c71d581978,0.17
68975,AutoAnything,10001-auto-part-base-replace-3y,30.99,30.8023,36.285714,35.99,5.0,f268d85e-79ad-44e4-9fda-3d780a6d6637,0.17


In [20]:
df_anom1['seller_name'].value_counts().head(15)

VORO MOTORS, Inc.         383
AromaTech Inc.            380
DailySale                 296
Brilliant Earth           259
JBL-US                    258
AutoAnything              246
Shop Indoor Golf          141
Motosport                 132
MoissaniteCo              122
Renogy                    111
Backcountry.com           100
Blackstone Products        98
Woojer                     96
ZVOX Audio                 86
Carandtruckremotes.com     78
Name: seller_name, dtype: int64

## Filter out some cases where there's a discpreancy between plan_purchase_price and raw_price:
- `plan_purchase_price` is closer to `target_price` rather than `min_price`
- `plan_purchase_price` > than both `min_price` and `target_price`

In [21]:
df_anom = df_pf[(df_pf['retail_target_diff'] < df_pf['retail_raw_rounded_diff']) & (df_pf['retail_raw_rounded_diff'] > 5)].sort_values(by='min_target_diff', ascending=False)
df_anom[price_col+['product_purchase_price', 'price', 'plan_sku', 'store_id']].head()

,seller_name,plan_id,plan_purchase_price,target_price,min_price_0.1,raw_price_10_rounded,product_purchase_price,price,plan_sku,store_id
276485,"VORO MOTORS, Inc.",10001-electric-mobility-base-replace-3y,299.0,395.8200,778.566667,799.0,2199.00,2199.00,EXTELS77,194b316d-c59c-4db1-b19e-347ebe7b182b
41657,"VORO MOTORS, Inc.",10001-electric-mobility-base-replace-3y,299.0,269.8200,588.233333,599.0,1499.00,1499.00,EXTELS76,194b316d-c59c-4db1-b19e-347ebe7b182b
66969,Shop Indoor Golf,10001-misc-elec-adh-replace-3y,649.0,609.6974,883.366667,899.0,2344.99,2344.99,EXTRPL195,fdbc2bfb-52b9-4d2f-967a-3f690fd99a38
66978,Shop Indoor Golf,10001-misc-elec-adh-replace-3y,649.0,609.6974,883.366667,899.0,2344.99,2344.99,EXTRPL195,fdbc2bfb-52b9-4d2f-967a-3f690fd99a38
59654,Shop Indoor Golf,10001-misc-elec-adh-replace-3y,549.0,518.7000,688.433333,699.0,1995.00,1995.00,EXTRPL194,fdbc2bfb-52b9-4d2f-967a-3f690fd99a38


In [22]:
df_anom['seller_name'].value_counts().head(15)

DailySale                 281
AutoAnything              241
JBL-US                    134
Motosport                 128
Shop Indoor Golf          122
MoissaniteCo              118
Backcountry.com            77
Competitive Cyclist        46
Jegs High Performance      33
RealTruck                  13
Zebit                      13
White Duck Outdoors        12
Renogy                     11
Evolve Skateboards USA      8
Balsam Hill                 5
Name: seller_name, dtype: int64

In [23]:
contracts_ex = df_anom['contract_id'].unique()

In [24]:
len(contracts_ex)

1279

In [25]:
df_pf_final = df_pf[~df_pf['contract_id'].isin(contracts_ex)]
len(df_pf_final)

65266

## Calculate Net Extend for different price floors - 0, 5, and 10%

In [ ]:
df_price['month_year']

In [41]:
(df_price['plan_transaction_date'].dt.month + 

0         5
1         7
2         9
3         8
4         9
         ..
372835    9
372836    9
372837    9
372838    9
372839    9
Name: plan_transaction_date, Length: 241954, dtype: int64

In [26]:
# calculate min price for different price floors
# 10%
df_price['raw_price_rounded'] = df_price['raw_price_0.1'].apply(round_price)

# 5%
df_price = calc_min_price(df_price, 0.05)
df_price = find_price_floor(df_price, 0.05)
df_price['raw_price_05_rounded'] = df_price['raw_price_0.05'].apply(round_price)

# 0%
df_price = calc_min_price(df_price, 0.0)
df_price = find_price_floor(df_price, 0.0)
df_price['raw_price_00_rounded'] = df_price['raw_price_0.0'].apply(round_price)

In [27]:
len(df_price[df_price['price_floor_0.1']==1])

66545

In [28]:
len(df_price[df_price['price_floor_0.05']==1])

37227

In [29]:
len(df_price[df_price['price_floor_0.0']==1])

28697

In [30]:
# Calculate merchant margin for each scenario
df_price['merch_margin'] = df_price.apply(lambda x: merch_margin(x, 'plan_purchase_price'), axis=1)
df_price['merch_margin_10'] = df_price.apply(lambda x: merch_margin(x, 'raw_price_rounded'), axis=1)
df_price['merch_margin_05'] = df_price.apply(lambda x: merch_margin(x, 'raw_price_05_rounded'), axis=1)
df_price['merch_margin_00'] = df_price.apply(lambda x: merch_margin(x, 'raw_price_00_rounded'), axis=1)

In [31]:
# Calculate net extend for each scenario
df_price['net_extend'] = df_price['plan_purchase_price'] - df_price['premium'] - df_price['merch_margin']
df_price['net_extend_10'] = df_price['raw_price_rounded'] - df_price['premium'] - df_price['merch_margin_10']
df_price['net_extend_05'] = df_price['raw_price_05_rounded'] - df_price['premium'] - df_price['merch_margin_05']
df_price['net_extend_00'] = df_price['raw_price_00_rounded'] - df_price['premium'] - df_price['merch_margin_00']

In [32]:
len(df_price[df_price['net_extend'] < 0])

595

In [33]:
len(df_price[df_price['net_extend_10'] < 0])

0

In [34]:
len(df_price[df_price['net_extend_05'] < 0])

9

In [35]:
df_price[df_price['net_extend_05'] < 0]['seller_name'].value_counts()

designeroptics.com    9
Name: seller_name, dtype: int64

In [36]:
len(df_price[df_price['net_extend_00'] < 0]['seller_name'].value_counts())

39

In [37]:
df_price[df_price['net_extend_00'] < 0]['seller_name'].value_counts()

Skullcandy                    1454
L'ange Hair                    138
Brilliant Earth                 62
Motosport                       58
BackfireBoardsUSA               40
Backcountry.com                 37
Med Mart                        32
T3                              25
RealTruck                       25
Competitive Cyclist             24
PCLiquidations                  20
Logitech                        19
JBL-US                          16
Impressions Vanity Co.          16
Senior.com                      15
designeroptics.com               9
Discount Ramps                   6
MoissaniteCo                     6
infanttech                       4
BB Wheels                        4
Jegs High Performance            4
AutoAnything                     4
MDMaxx                           4
THE SMARTY CO.                   4
Steep and Cheap                  3
Pitbull Audio                    3
HA-US                            3
Jenson USA                       3
DailySale           

In [38]:
df_merch_net_extend = df_price.groupby(by='seller_name').sum()[['net_extend', 'net_extend_10', 'net_extend_05', 'net_extend_00']].reset_index()
df_merch_net_extend.head()

,seller_name,net_extend,net_extend_10,net_extend_05,net_extend_00
0,"PowerX Technology, Inc",223.4295,207.1795,199.3795,195.4795
1,1MOREUSA,11744.8125,8724.4125,8170.8375,7779.3375
2,AIR RELAX,13474.5270,15247.4730,15247.3330,15247.3330
3,Action Clutch,1862.5290,1765.0290,1765.0290,1765.0290
4,Active Lifestyle Store,760.0890,505.4990,459.7190,422.8990


In [39]:
df_merch_net_extend.sort_values(by='net_extend', ascending=False)

,seller_name,net_extend,net_extend_10,net_extend_05,net_extend_00
195,RealTruck,429111.992,537835.6210,527907.3110,522077.1440
104,Jegs High Performance,168290.192,181792.0640,180856.2440,180355.2380
23,Balsam Hill,164178.162,261080.8020,261068.8020,261068.8020
265,Xebec,102066.273,98125.0000,98125.0000,98125.0000
38,Capillus,99443.102,95645.1020,95645.1020,95645.1020
...,...,...,...,...,...
109,JumpSport,0.000,0.0000,0.0000,0.0000
278,iPCstore,0.000,0.0000,0.0000,0.0000
30,Bose,-7.280,688.8475,688.8475,688.8475
232,The Natural Sapphire Company,-252.470,280.0300,280.0300,280.0300


In [92]:
col_list = ['plan_purchase_price', 'min_price_0.1', 'target_price', 'raw_price_rounded']
merchant = 'Balsam Hill'
df_price[(df_price['seller_name']==merchant) & (df_price['plan_purchase_price'] < df_price['raw_price_rounded'])][col_list]

,plan_purchase_price,min_price_0.1,target_price,raw_price_rounded
14188,429.00,268.72,571.74,549.00
14189,299.00,179.16,395.82,399.00
14190,379.00,301.90,395.82,399.00
14191,27.99,10.16,29.90,29.99
14192,35.99,11.60,53.82,54.99
...,...,...,...,...
341046,17.98,5.80,29.90,29.99
341047,39.99,20.32,53.82,54.99
341048,17.98,5.80,29.90,29.99
341049,21.99,10.16,29.90,29.99


In [76]:
df_merch_net_extend[df_merch_net_extend['seller_name']=='Jetson Electric Bikes']

,seller_name,net_extend,net_extend_10,net_extend_05,net_extend_00
105,Jetson Electric Bikes,38133.0425,46707.511,43207.651,40133.97


## Impact Analysis
- how large is the difference between min_price and target_price?
- which merchants are most impacted?

In [38]:
df_pf_final['seller_name'].value_counts().head(20)

BlendJet                  16284
JBL-US                    10688
RealTruck                  3947
Carandtruckremotes.com     3862
L'ange Hair                3755
Skullcandy                 2644
EightVape                  2159
RaceDayQuads               1376
DailySale                  1341
Parts Dr                   1283
Jetson Electric Bikes      1129
Logitech                   1031
1MOREUSA                   1005
Jegs High Performance       887
Wyze Ecommerce              798
PCLiquidations              778
Motosport                   761
Blackstone Products         633
Renogy                      562
AromaTech Inc.              551
Name: seller_name, dtype: int64

In [45]:
# In most cases, difference between target and minimum price isn't significantly differents

In [40]:
df_pf_final['min_target_diff'].describe()

count    65131.000000
mean         5.360932
std         18.913240
min          0.000400
25%          0.384900
50%          0.970231
75%          3.534000
max        449.350000
Name: min_target_diff, dtype: float64

In [41]:
df_pf_final[df_pf_final['min_target_diff'] < 5] #['seller_name'].value_counts().head(10)

,contract_id,store_id,seller_name,plan_id,product_purchase_price,plan_purchase_price,premium,merchant_margin_usd,plan_sku,contract_status,...,price_diff_pct,raw_price_0.1,price_floor_0.1,retail_min_diff,retail_target_diff,min_target_diff,min_target,target_diff,raw_price_10_rounded,retail_raw_rounded_diff
6,052e03cd-fc49-43b3-9383-5bad1bce6500,0233ca5d-1b3f-4983-8b37-e3c503572f5a,T3,B0-HBGEN-2y,149.99,22.99,9.18,11.495,EXTRPLBG138,live,...,0.019673,22.950,1,0.040,0.4915,0.4515,0.153010,0.003010,22.99,0.0
7,b11738af-21a5-4a83-b9c1-5d6920dd5fc3,0233ca5d-1b3f-4983-8b37-e3c503572f5a,T3,B0-HBGEN-2y,149.99,22.99,9.18,11.495,EXTRPLBG138,live,...,0.019673,22.950,1,0.040,0.4915,0.4515,0.153010,0.003010,22.99,0.0
8,4876d468-0a74-4ed2-a03a-5b1251643fad,0233ca5d-1b3f-4983-8b37-e3c503572f5a,T3,B0-HBGEN-3y,129.99,34.99,13.73,17.495,EXTRPLBG256,live,...,0.126060,34.325,1,0.665,4.9920,4.3270,0.228849,0.028849,33.99,1.0
10,d7e9a3c5-7a55-4d55-bfb0-ffc3708cfed3,0233ca5d-1b3f-4983-8b37-e3c503572f5a,T3,B0-HBGEN-3y,179.99,54.99,20.01,27.495,EXTRPLBG265,live,...,0.060470,50.025,1,4.965,7.9900,3.0250,0.212872,0.012872,49.99,5.0
18,3d369850-4ba9-42da-a196-a28753482e0d,0233ca5d-1b3f-4983-8b37-e3c503572f5a,T3,B0-HBGEN-3y,179.99,54.99,20.01,27.495,EXTRPLBG265,live,...,0.060470,50.025,1,4.965,7.9900,3.0250,0.212872,0.012872,49.99,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
371434,27c0f045-c17f-4377-94d4-c745cff5b97f,24ece812-3f37-445c-ab71-018222639114,PADO,A0-HBGEN-3y,99.95,26.99,10.52,13.495,EXTRPLAG251,live,...,0.011787,26.300,1,0.690,1.0000,0.3100,0.202386,0.002386,25.99,1.0
371439,93a01775-9fd5-456a-9004-c524229cede1,24ece812-3f37-445c-ab71-018222639114,PADO,A0-HBGEN-3y,99.95,26.99,10.52,13.495,EXTRPLAG251,live,...,0.011787,26.300,1,0.690,1.0000,0.3100,0.202386,0.002386,25.99,1.0
371441,62bbd0e2-7daf-4e84-ba6d-e476e2c2f7e2,24ece812-3f37-445c-ab71-018222639114,PADO,A0-HBGEN-3y,99.95,26.99,10.52,13.495,EXTRPLAG251,live,...,0.011787,26.300,1,0.690,1.0000,0.3100,0.202386,0.002386,25.99,1.0
371443,69219dae-56be-49fb-9372-bfd56d987f07,24ece812-3f37-445c-ab71-018222639114,PADO,A0-HBGEN-3y,99.95,25.99,10.52,12.995,EXTRPLAG251,live,...,0.011787,26.300,1,0.310,0.0000,0.3100,0.202386,0.002386,25.99,0.0


In [42]:
df_jetson = df_price[df_price['seller_name']=='Jetson Electric Bikes']
df_jetson[price_col]

,seller_name,plan_id,plan_purchase_price,target_price,min_price_0.1
7048,Jetson Electric Bikes,10001-scooter-base-replace-2y,74.99,16.9990,74.890909
7049,Jetson Electric Bikes,10001-scooter-base-replace-2y,74.99,16.9990,74.890909
8475,Jetson Electric Bikes,10001-electric-mobility-base-replace-2y,74.99,20.9986,74.890909
8476,Jetson Electric Bikes,10001-electric-mobility-base-replace-2y,74.99,20.9986,74.890909
8477,Jetson Electric Bikes,10001-electric-mobility-base-replace-1y,49.99,14.9990,49.909091
...,...,...,...,...,...
363472,Jetson Electric Bikes,10001-electric-mobility-base-replace-1y,49.99,22.9990,49.909091
363473,Jetson Electric Bikes,10001-electric-mobility-base-replace-1y,49.99,22.9990,49.909091
363474,Jetson Electric Bikes,10001-electric-mobility-base-replace-1y,49.99,22.9990,49.909091
363475,Jetson Electric Bikes,10001-electric-mobility-base-replace-1y,49.99,19.9990,49.909091
